In [1]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')

nltk.download('stopwords')
!pip install bpemb

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
     |████████████████████████████████| 1.1MB 8.1MB/s 


In [2]:
import os
import numpy
import pandas as pd
import re


In [3]:
def clean_text_doc(text):
    stpwrds=[]
    unwanted=['ـ','_','"','！',"'",'=','@','&','%','.',',',':','\\','$','^','<','>','!','?','{','}',';','\n','\t','(',')','[',']','/','*','+','#','\u200c','\ufeff','-','_','|']
    wanted=['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H','I','J','K','L','M','N','O','P','Q','R','S','T','U','V','W','X','Y','Z','a','b','c','d','e','f','g','h','i','j','k','l','m','n','o',
 'p','q','r','s','t','u','v','w','x','y','z']
    for i in unwanted:
        text = text.replace(i,'')
    sent=[]
    text=text.split(' ')
    
    for word in text:
        
        if word not in stpwrds and word.isalpha():
            
            sent.append(word)
    return ' '.join(sent)

In [4]:
data=pd.read_csv('/content/drive/My Drive/SentimentFire/malayalam_train.tsv',sep='\t')

test=pd.read_csv('/content/drive/My Drive/SentimentFire/malayalam_test - malayalam_test.tsv',sep='\t')

data['text']=data['text'].apply(lambda x:clean_text_doc(x))
test['text']=test['text'].apply(lambda x:clean_text_doc(x))

In [5]:
test.head()

,id,text
0,ml_sen_1,Bollywood film Newton inte remake aano
1,ml_sen_2,endukond viewrs koodunnilla ippozhum aayitollu
2,ml_sen_3,Mara paazhu mega mairananil ninnum ethil koodu...
3,ml_sen_4,Video nay cang xem cang thit
4,ml_sen_5,Sunny chechiye kaanan vannathu njan maathram aano


In [6]:
>>> from bpemb import BPEmb
# load English BPEmb model with default vocabulary size (10k) and 50-dimensional embeddings
>>> bpemb_en = BPEmb(lang="en",vs=10000)

downloading https://nlp.h-its.org/bpemb/en/en.wiki.bpe.vs10000.model


100%|██████████| 400869/400869 [00:00<00:00, 598192.67B/s]


downloading https://nlp.h-its.org/bpemb/en/en.wiki.bpe.vs10000.d100.w2v.bin.tar.gz


100%|██████████| 3784656/3784656 [00:01<00:00, 3173473.68B/s]
/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:254: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [27]:
bpemb_en.encode('Indiayile friends')

['▁ind', 'i', 'ay', 'ile', '▁friends']

In [7]:
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
import warnings
warnings.filterwarnings(action='ignore')
import pandas as pd
import gensim
from gensim.models import Word2Vec
import numpy as np
import csv
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [8]:
texts=[]
for i in data.text:
    temp=[]
    for j in bpemb_en.encode(i):
      if j.startswith('▁'):j=j[1:]
      temp.append(j.lower())
    texts.append(temp)

In [22]:
texts[9]

['i', 'a', 'm', 'a', 'k', 'att', 'a', 'mam', 'mo', 'ok', 'ka', 'fan']

In [21]:
data.text[9]

'I a m a katta mammookka fan'

In [ ]:
#using skip gram
w2v = gensim.models.Word2Vec(texts, min_count = 1, size = 100, window = 3,sg=1) 

In [ ]:
vocab=list(w2v.wv.vocab)

In [ ]:
embeddings_index = {}
for line in data.text:
    words = bpemb_en.encode(line)
    for word in words:
      vec=[]
      if word.startswith('▁'):word=word[1:]
      word=word.lower()
      vec.extend(w2v[word])
        
      embeddings_index[word] =vec

In [ ]:
# The maximum number of words to be used. (most frequent)
MAX_NB_WORDS = 50000
# Max number of words in each complaint.
MAX_SEQUENCE_LENGTH = 250
# This is fixed.
EMBEDDING_DIM = 100

tokenizer = Tokenizer(num_words=MAX_NB_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)
tokenizer.fit_on_texts(data['text'].values)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 12628 unique tokens.


In [ ]:
X = tokenizer.texts_to_sequences(data['text'].values)
X = pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', X.shape)

Shape of data tensor: (4851, 250)


In [ ]:
Y = pd.get_dummies(data.category).values
print('Shape of label tensor:', Y.shape)

Shape of label tensor: (4851, 5)


In [ ]:
X_train,Y_train=X,Y

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D,Bidirectional
from keras.callbacks import EarlyStopping

In [ ]:
model = Sequential()
model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=X.shape[1]))
model.add(SpatialDropout1D(0.2))
model.add(Bidirectional(LSTM(100, dropout=0.2, recurrent_dropout=0.2)))
model.add(Dense(5, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 250, 200)          10000000  
_________________________________________________________________
spatial_dropout1d_2 (Spatial (None, 250, 200)          0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 200)               240800    
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 1005      
Total params: 10,241,805
Trainable params: 10,241,805
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
epochs = 10
batch_size = 128

history = model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size,validation_split=0.1,callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])

Epoch 1/10
35/35 [==============================] - 67s 2s/step - loss: 1.4089 - accuracy: 0.4165 - val_loss: 1.3314 - val_accuracy: 0.4527
Epoch 2/10
35/35 [==============================] - 66s 2s/step - loss: 1.1370 - accuracy: 0.5734 - val_loss: 1.0408 - val_accuracy: 0.6152
Epoch 3/10
35/35 [==============================] - 66s 2s/step - loss: 0.6583 - accuracy: 0.7826 - val_loss: 0.9137 - val_accuracy: 0.6667
Epoch 4/10
35/35 [==============================] - 66s 2s/step - loss: 0.3458 - accuracy: 0.8788 - val_loss: 0.9648 - val_accuracy: 0.6584
Epoch 5/10
35/35 [==============================] - 67s 2s/step - loss: 0.1892 - accuracy: 0.9430 - val_loss: 1.0429 - val_accuracy: 0.6646
Epoch 6/10
35/35 [==============================] - 67s 2s/step - loss: 0.1096 - accuracy: 0.9704 - val_loss: 1.1580 - val_accuracy: 0.6564


In [ ]:
epochs = 10
batch_size = 64

history = model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size,validation_split=0.1,callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])

Epoch 1/10
69/69 [==============================] - 132s 2s/step - loss: 0.0773 - accuracy: 0.9778 - val_loss: 1.2306 - val_accuracy: 0.6728
Epoch 2/10
69/69 [==============================] - 132s 2s/step - loss: 0.0598 - accuracy: 0.9805 - val_loss: 1.3281 - val_accuracy: 0.6420
Epoch 3/10
69/69 [==============================] - 131s 2s/step - loss: 0.0412 - accuracy: 0.9869 - val_loss: 1.4239 - val_accuracy: 0.6296
Epoch 4/10
69/69 [==============================] - 133s 2s/step - loss: 0.0275 - accuracy: 0.9908 - val_loss: 1.3973 - val_accuracy: 0.6523


In [ ]:
epochs = 10
batch_size = 32

history = model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size,validation_split=0.1,callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])

Epoch 1/10
137/137 [==============================] - 264s 2s/step - loss: 0.0308 - accuracy: 0.9906 - val_loss: 1.5375 - val_accuracy: 0.6440
Epoch 2/10
137/137 [==============================] - 264s 2s/step - loss: 0.0319 - accuracy: 0.9897 - val_loss: 1.5294 - val_accuracy: 0.6502
Epoch 3/10
137/137 [==============================] - 265s 2s/step - loss: 0.0212 - accuracy: 0.9940 - val_loss: 1.7260 - val_accuracy: 0.6337
Epoch 4/10
137/137 [==============================] - 262s 2s/step - loss: 0.0182 - accuracy: 0.9940 - val_loss: 2.1008 - val_accuracy: 0.6049
Epoch 5/10
137/137 [==============================] - 261s 2s/step - loss: 0.0191 - accuracy: 0.9922 - val_loss: 2.0402 - val_accuracy: 0.6091


In [ ]:
epochs = 10
batch_size = 16

history = model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size,validation_split=0.1,callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])

Epoch 1/10
273/273 [==============================] - 518s 2s/step - loss: 0.0402 - accuracy: 0.9872 - val_loss: 1.5963 - val_accuracy: 0.6379
Epoch 2/10
273/273 [==============================] - 517s 2s/step - loss: 0.0293 - accuracy: 0.9895 - val_loss: 1.8612 - val_accuracy: 0.6255
Epoch 3/10
273/273 [==============================] - 513s 2s/step - loss: 0.0228 - accuracy: 0.9924 - val_loss: 1.6752 - val_accuracy: 0.6317
Epoch 4/10
273/273 [==============================] - 516s 2s/step - loss: 0.0151 - accuracy: 0.9954 - val_loss: 1.7863 - val_accuracy: 0.6317


In [ ]:
def prediction(text):

  seq = tokenizer.texts_to_sequences([text])
  padded = pad_sequences(seq, maxlen=MAX_SEQUENCE_LENGTH)
  pred = model.predict(padded)
  
  labels = ['Mixed_feelings ','Negative ','Positive ','not-malayalam ','unknown_state ']

  return labels[np.argmax(pred)]

In [ ]:
spreds=[]
for txt in test.text.values:
  spreds.append(prediction(txt))

In [ ]:
set(spreds)

{'Mixed_feelings ',
 'Negative ',
 'Positive ',
 'not-malayalam ',
 'unknown_state '}

In [ ]:
#set(test.category)

In [ ]:
#from sklearn.metrics import classification_report
#print(classification_report(spreds,test.category))

In [ ]:
preds=[]
for txt in data.text.values:
  preds.append(prediction(txt))

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(preds,data.category))

                 precision    recall  f1-score   support

Mixed_feelings        0.91      0.92      0.92       286
      Negative        0.95      0.97      0.96       541
      Positive        0.96      0.97      0.97      1995
 not-malayalam        0.97      0.93      0.95       673
 unknown_state        0.96      0.96      0.96      1356

       accuracy                           0.96      4851
      macro avg       0.95      0.95      0.95      4851
   weighted avg       0.96      0.96      0.96      4851



In [ ]:
model_save_path = "subwords_model.h5"
model.save(model_save_path)

In [ ]:


import os
import numpy
import pandas as pd
import re
import glob
from nltk.tokenize import word_tokenize
from nltk import FreqDist
from nltk import trigrams
from nltk import bigrams
from nltk import ngrams
from nltk.collections import defaultdict
from nltk.classify.scikitlearn import SklearnClassifier
from sklearn.linear_model import LogisticRegression
from gensim.models import KeyedVectors
from collections import Counter 
from nltk.classify.util import accuracy
from sklearn.metrics import precision_score, recall_score, f1_score, classification_report,accuracy_score
import pandas as pd
import nltk
import gensim
import collections

In [ ]:
from sklearn.naive_bayes import MultinomialNB
mnb=SklearnClassifier(MultinomialNB())
mnb.train(train_feats)

pred1=[]
for txt in test['text']:
    txt=BOW(txt.split(' '))
    pred1.append(mnb.classify(txt))
print(classification_report(pred1,test['category']))

In [ ]:
#ngram-mlp

In [ ]:
import unicodedata
from string import punctuation
from nltk.tokenize import WhitespaceTokenizer
from nltk import ngrams


In [ ]:
def get_word_ngrams(tokens, n):
    ngrams_list = []
    ngrams_list.append(list(ngrams(tokens, n)))
    ngrams_flat_tuples = [ngram for ngram_list in ngrams_list for ngram in ngram_list]
    format_string = '%s'
    for i in range(1, n):
        format_string += (' %s')
    ngrams_list_flat = [format_string % ngram_tuple for ngram_tuple in ngrams_flat_tuples]
    return ngrams_list_flat

def get_char_ngrams(word, n):
    ngrams_list = []
    word = re.sub(r'ς', 'σ', word)
    ngrams_list.append(list(ngrams(word, n, pad_left=True, pad_right=True, left_pad_symbol='_', right_pad_symbol='_')))
    
    # Removing redundant ngrams:
    if (n > 2):
        redundant_combinations = n - 2
        ngrams_list = [ngram_list[redundant_combinations : -redundant_combinations] for ngram_list in ngrams_list]
    
    ngrams_flat_tuples = [ngram for ngram_list in ngrams_list for ngram in ngram_list]
    format_string = ''
    for i in range(0, n):
        format_string += ('%s')
    ngrams_list_flat = [format_string % ngram_tuple for ngram_tuple in ngrams_flat_tuples]
    return ngrams_list_flat

# Feature extractor
def get_ngram_features(sent): # The reason I do not use NLTK's everygrams to extract the features quickly is because the behavior of my n-gram extractor is modified to remove redundant n-grams. Also, I need to label word and char n-grams to avoid ambiguity
    sentence_tokens = WhitespaceTokenizer().tokenize(sent)
    
    features = {}
    
    # Word unigrams
    ngrams = get_word_ngrams(sentence_tokens, 1)
    for ngram in ngrams:
        features[f'word({ngram})'] = features.get(f'word({ngram})', 0) + 1 # The second parameter to .get() is a default value if the key doesn't exist.
    
    # Word bigrams
    ngrams = get_word_ngrams(sentence_tokens, 2)
    for ngram in ngrams:
        features[f'word_bigram({ngram})'] = features.get(f'word_bigram({ngram})', 0) + 1
        
    # Word trigrams
    ngrams = get_word_ngrams(sentence_tokens, 2)
    for ngram in ngrams:
        features[f'word_trigram({ngram})'] = features.get(f'word_trigram({ngram})', 0) + 1
    
    # Char unigrams
    for word in sentence_tokens:
        ngrams = get_char_ngrams(word, 1)
        for ngram in ngrams:
            features[f'char({ngram})'] = features.get(f'char({ngram})', 0) + 1
    
    # Char bigrams
    for word in sentence_tokens:
        ngrams = get_char_ngrams(word, 2)
        for ngram in ngrams:
            features[f'char_bigram({ngram})'] = features.get(f'char_bigram({ngram})', 0) + 1
    
    # Char trigrams
    for word in sentence_tokens:
        ngrams = get_char_ngrams(word, 3)
        for ngram in ngrams:
            features[f'char_trigram({ngram})'] = features.get(f'char_trigram({ngram})', 0) + 1
            
            # Char trigrams
    for word in sentence_tokens:
        ngrams = get_char_ngrams(word, 4)
        for ngram in ngrams:
            features[f'char_4gram({ngram})'] = features.get(f'char_4gram({ngram})', 0) + 1
            
    for word in sentence_tokens:
        ngrams = get_char_ngrams(word, 5)
        for ngram in ngrams:
            features[f'char_5gram({ngram})'] = features.get(f'char_5gram({ngram})', 0) + 1
    
    return features

In [ ]:
train_set_sents = data.text
train_set_labels = data.category
test_set_sents =test.text
#test_set_labels = test.category

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

count_vect = CountVectorizer(analyzer=get_ngram_features)

train_set_vectors = count_vect.fit_transform(train_set_sents)
test_set_vectors = count_vect.transform(test_set_sents) # Unlike fit_transform(), transform() does not change the count vectorizer's vocabulary so it should be used for the test set.
train_set_vectors

<4851x134433 sparse matrix of type '<class 'numpy.int64'>'
	with 946340 stored elements in Compressed Sparse Row format>

In [ ]:
from sklearn.neural_network import MLPClassifier

clf_mlp = MLPClassifier(hidden_layer_sizes=(150,100,50), max_iter=300,activation = 'relu',solver='adam',random_state=1)# Again, dual = True. Default solver = 'liblinear'. It's recommended for smaller databases. For bigger databases, 'saga' could be used.
clf_mlp.fit(train_set_vectors, train_set_labels)

pred2 = clf_mlp.predict(test_set_vectors)

#print('\t\t\tPERFORMANCE\n')
#print('Accuracy:', round(accuracy_score(test_set_labels, pred2), 2), '\n')

#print(classification_report(test_set_labels, pred2))

In [ ]:
#w2v+ml
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
import warnings
warnings.filterwarnings(action='ignore')
import pandas as pd
import gensim
from gensim.models import Word2Vec

txt=[]
for i in data.text:
    temp=[]
    for j in i.split(' '):
        temp.append(j.lower())
    txt.append(temp)

#using skip gram
w2v = gensim.models.Word2Vec(txt, min_count = 1, size = 100, window = 3,sg=1) 

In [ ]:
vocab=list(w2v.wv.vocab)

def feats(txt):
    ff=[]
    for word in txt.split(' '):
        try:
            ff.extend(w2v[word.lower()])
        except: ff.extend([0.0])
    return ff

In [ ]:
train_set_sents = data.text
train_set_labels = data.category
test_set_sents =test.text
#test_set_labels = test.category

from sklearn.feature_extraction.text import CountVectorizer

count_vect = CountVectorizer(analyzer=feats)

train_set_vectors = count_vect.fit_transform(train_set_sents)
test_set_vectors = count_vect.transform(test_set_sents) # Unlike fit_transform(), transform() does not change the count vectorizer's vocabulary so it should be used for the test set.
train_set_vectors

<4851x1251687 sparse matrix of type '<class 'numpy.int64'>'
	with 3633684 stored elements in Compressed Sparse Row format>

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

clf_multinomialNB = MultinomialNB() # There are no params for MultinomialDB that prevent overfitting, so any overfitting is caused by the small dataset size.
clf_multinomialNB.fit(train_set_vectors, train_set_labels)

pred3 = clf_multinomialNB.predict(test_set_vectors)

#print('\t\t\tPERFORMANCE\n')
#print('Accuracy:', round(accuracy_score(test_set_labels, pred2), 2), '\n')

#print(classification_report(test_set_labels, pred3))

In [ ]:
predf=[]
for i in range(len(pred2)):
  print([pred2[i],pred3[i],spreds[i]])
  occurence_count = Counter([pred2[i],pred3[i],spreds[i]])
  pred = occurence_count.most_common(1)[0][0]
  predf.append(pred)

['unknown_state ', 'unknown_state ', 'Mixed_feelings ']
['unknown_state ', 'unknown_state ', 'Positive ']
['Mixed_feelings ', 'Positive ', 'not-malayalam ']
['unknown_state ', 'unknown_state ', 'not-malayalam ']
['Positive ', 'unknown_state ', 'unknown_state ']
['unknown_state ', 'unknown_state ', 'unknown_state ']
['Positive ', 'Positive ', 'Positive ']
['Negative ', 'Negative ', 'Negative ']
['Positive ', 'unknown_state ', 'Positive ']
['Negative ', 'Negative ', 'Negative ']
['unknown_state ', 'Positive ', 'unknown_state ']
['not-malayalam ', 'not-malayalam ', 'not-malayalam ']
['not-malayalam ', 'Positive ', 'not-malayalam ']
['Positive ', 'Positive ', 'Positive ']
['unknown_state ', 'unknown_state ', 'Positive ']
['unknown_state ', 'Positive ', 'Positive ']
['unknown_state ', 'unknown_state ', 'unknown_state ']
['Positive ', 'Positive ', 'Positive ']
['Positive ', 'unknown_state ', 'unknown_state ']
['Positive ', 'Positive ', 'unknown_state ']
['Positive ', 'Positive ', 'Positive '

In [ ]:
#print(classification_report(test.category,predf))

                 precision    recall  f1-score   support

Mixed_feelings        0.62      0.11      0.19        44
      Negative        0.67      0.47      0.55        51
      Positive        0.67      0.83      0.74       224
 not-malayalam        0.78      0.83      0.81        60
 unknown_state        0.73      0.71      0.72       161

       accuracy                           0.70       540
      macro avg       0.70      0.59      0.60       540
   weighted avg       0.70      0.70      0.68       540



In [ ]:
test.tail(10)

,id,text
1338,ml_sen_1339,Lalettan paranjal kanathe pattume Lalettan ishtam
1339,ml_sen_1340,Taken Enna movie pole aavum enn thonnunnu
1340,ml_sen_1341,MINUTES MARANAMASS TRAILER KATTA WAITING MANJU...
1341,ml_sen_1342,Mammoonny Komali Loka tholvi Sammathikkunnavar...
1342,ml_sen_1343,Enta ponno eppolu ntha oru ethu
1343,ml_sen_1344,views and likes
1344,ml_sen_1345,Nasik doll tiltle bgm aane highlight
1345,ml_sen_1346,Mohanlalile ishtapetta vekthiyayirunnu njan pa...
1346,ml_sen_1347,ivaruku ivlo age nu sonna evan nambuvan
1347,ml_sen_1348,Kanditt Amala Paul Aadai tamil mattoru version...


In [ ]:
set(predf)

{'Mixed_feelings ',
 'Negative ',
 'Positive ',
 'not-malayalam ',
 'unknown_state '}

In [ ]:
len(predf)

1348

In [ ]:
len(test)

1348

In [ ]:
outdf=pd.DataFrame()
outdf['text']=test.text
outdf['label']=predf

In [ ]:
outdf.head()

,text,label
0,Bollywood film Newton inte remake aano,unknown_state
1,endukond viewrs koodunnilla ippozhum aayitollu,unknown_state
2,Mara paazhu mega mairananil ninnum ethil koodu...,Mixed_feelings
3,Video nay cang xem cang thit,unknown_state
4,Sunny chechiye kaanan vannathu njan maathram aano,unknown_state


In [ ]:
set(outdf.label)

{'Mixed_feelings ',
 'Negative ',
 'Positive ',
 'not-malayalam ',
 'unknown_state '}

In [ ]:

outdf.to_csv('/content/drive/My Drive/SentimentFire/outputs/MUCS_malayalam.tsv',sep='\t')